## Working with an LLM programmatically

ChatGPT のような大規模言語モデル (LLM) をこれまでに操作したことがあるはずです。これは通常、UI またはアプリケーションを介して行われます。

このノートブックでは、Python を使用して、LLM の API を介して直接接続し、クエリを実行します。このラボでは、モデル **Granite-7B-Instruct** (https://huggingface.co/ibm-granite/granite-7b-instruct) を選択しました。これは、IBM Research によって開発された完全にオープン ソースのモデル (Apache 2.0 ライセンス) です。

このモデルは、小さいモデルであっても、実行するには 24 GB の RAM を備えた GPU が必要なため、ラボ クラスターにすでにデプロイされています...

### 要件とインポート

ラボの指示に従って起動する適切なワークベンチ イメージを選択した場合は、必要なライブラリがすべてすでに用意されているはずです。そうでない場合は、次のセルの最初の行のコメントを解除して、適切なパッケージをすべてインストールします。その後、必要なライブラリをインポートします。

In [ ]:
# 適切なワークベンチ イメージを選択していない場合、またはこのノートブックをワークショップ環境外で使用している場合にのみ、次の行のコメントを解除します。
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain_community.llms import VLLMOpenAI

### Langchain

Langchain (https://www.langchain.com/) は、言語モデルを利用したアプリケーションを開発するためのフレームワークです。LLM API を適切にクエリするために手動で記述する必要があるすべての定型コードを処理します。

まず、LLM API をクエリできる場所とモデルに適用されるいくつかのパラメータで定義される **llm** インスタンスを作成します。たとえば、`max_new_tokens` は、モデルに最大 512 個のトークン (単語または単語の一部) で応答するように指示します。ここで非常に低く設定されている `temperature` は、モデルに真実に基づいたままで、あまり「創造的」にならないように指示します。結局のところ、ここでは派手な詩を書こうとしているわけではありません。

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://granite-7b-instruct-predictor.ic-shared-llm.svc.cluster.local:8080"

# LLM definition
llm = VLLMOpenAI(           # 私たちは vLLM OpenAI 互換 API クライアントを使用しています。ただし、モデルは OpenAI ではなく OpenShift AI 上で実行されています。
    openai_api_key="EMPTY",   # そのため、これには OpenAI キーは必要ありません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="granite-7b-instruct",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

また、モデルに送信するすべてのリクエスト (「プロンプト」) に適用される **template** も必要です。

モデルにクエリを実行する場合、ユーザーが入力したものを直接送信することはほとんど望ましくありません。このエントリに加えて、モデルが処理方法を理解できるように、適切な指示をモデルに与える必要があります。つまり、何にどのように答えるか、何に答えてはいけないか、どのような口調で答えるかなどです...

In [ ]:
template="""<|system|>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".

<|user|>
### QUESTION:
{input}

### ANSWER:
<|assistant|>
"""
prompt = PromptTemplate(input_variables=["input"], template=template)

Langchain を使用すると、これらの要素を簡単に「つなぎ合わせて」、モデルのクエリに使用する **conversation** オブジェクトを作成できるようになりました。

In [ ]:
conversation = prompt | llm

これでモデルをクエリする準備ができました。

In [ ]:
query = "What is Artificial Intelligence?"

conversation.invoke(input=query); # 行末の";"は最終出力（ストリームされた回答の繰り返し）を非表示にします。

必要に応じて、セクション 3.7 のこのノートブックに戻って、オプションの演習を行うこともできます。